In [67]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../")
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import track1

backbone = generate_backbone(bpm,beat_per_bar,bar_per_emphasis, n_bars_buildup, progression)

handling {'part': 'intro', 'buildup': 1.0, 'duration': 4, 'tracks': [('base', 'no'), ('chords', 'no'), ('drums1', 'no'), ('drums2', 'no'), ('melody1', 'yes'), ('melody2', 'no')]}
handling {'part': 'verse', 'buildup': 1.0, 'duration': 12, 'tracks': [('base', 'yes'), ('chords', 'yes'), ('drums1', 'yes'), ('drums2', 'no'), ('melody1', 'yes'), ('melody2', 'no')]}
handling {'part': 'pause', 'buildup': 1.0, 'duration': 4, 'tracks': [('base', 'yes'), ('chords', 'no'), ('drums1', 'no'), ('drums2', 'no'), ('melody1', 'no'), ('melody2', 'no')]}
handling {'part': 'verse', 'buildup': 1.0, 'duration': 12, 'tracks': [('base', 'yes'), ('chords', 'yes'), ('drums1', 'yes'), ('drums2', 'yes'), ('melody1', 'yes'), ('melody2', 'yes')]}
handling {'part': 'verse', 'buildup': 1.0, 'duration': 6, 'tracks': [('base', 'yes'), ('chords', 'yes'), ('drums1', 'yes'), ('drums2', 'yes'), ('melody1', 'yes'), ('melody2', 'yes')]}
handling {'part': 'verse', 'buildup': 0.3, 'duration': 6, 'tracks': [('base', 'yes'), ('ch

In [70]:
drum_generator_1 = produce_drum_generator()
drum_generator_2 = produce_drum_generator()

In [ ]:
first_drum_1 = True
first_drum_2 = True


current_length = 0.0

for b in backbone:
    
    ###############
    #
    # Drums
    #
    
    t =  (1 - b['buildup_factor']) + b['buildup_factor']**2 * 2
    
    clonk = 49 if b["final_bar"] else 42
    
    nsteps = beat_per_bar * steps_per_beat 
    beams = 128

    ########
    # Drum 1
    #
    if b["drums1"] == 1.0:
        if first_drum_1:
            drum_seq = generate_one_bar_sequence(generator=drum_generator_1, qpm=bpm, temp=t, number_of_steps=nsteps, beam_size=beams,
                                                 primer = [(35,),(),(42,),(),(35,),(),(clonk,),()])
            
            retimed = music_pb2.NoteSequence()
            retimed.CopyFrom(drum_seq)

            for note in retimed.notes:
                note.start_time += current_length
                note.end_time += current_length
                retimed.total_time += current_length
                
            full_drum_seq1 = retimed
            
            first_drum_1 = False
            
            print("number of notes (drum_seq1) = " + str(len(full_drum_seq1.notes)))
            
        else:
            drum_seq = generate_one_bar_sequence(generator=drum_generator_1, qpm=bpm, temp=t, number_of_steps=nsteps, beam_size=beams,
                                                 primer = [(35,),(),(42,),(),(35,),(),(clonk,),()])
            full_drum_seq1 = adjust_sequence_times_and_merge(full_drum_seq1,drum_seq,current_length)
            print("number of notes (drum_seq1) = " + str(len(full_drum_seq1.notes)))
        

            
    ########
    # Drum 2
    #    
    if b["drums2"] == 1.0:
        if first_drum_2:
            drum_seq = generate_one_bar_sequence(generator=drum_generator_2, qpm=bpm, temp=t, number_of_steps=nsteps, beam_size=beams,
                                                 primer = [(46,),(42,),(46,42,),(42,),(46,),(42,),(46,clonk,),(42,)])
            
            retimed = music_pb2.NoteSequence()
            retimed.CopyFrom(drum_seq)

            for note in retimed.notes:
                note.start_time += current_length
                note.end_time += current_length
                retimed.total_time += current_length
                
            full_drum_seq2 = retimed
            
            first_drum_2 = False
            
            print("number of notes (drum_seq2) = " + str(len(full_drum_seq2.notes)))
            
        else:
            drum_seq = generate_one_bar_sequence(generator=drum_generator_2, qpm=bpm, temp=t, number_of_steps=nsteps, beam_size=beams,
                                                 primer = [(46,),(42,),(46,42,),(42,),(46,),(42,),(46,clonk,),(42,)])
            full_drum_seq2 = adjust_sequence_times_and_merge(full_drum_seq2,drum_seq,current_length)
            print("number of notes (drum_seq2) = " + str(len(full_drum_seq2.notes)))
            
    current_length += compute_time(bpm,nsteps,steps_per_beat)
    print("duration = " + str(current_length))

        
    ###############
    #
    # Tune
    #
    
    
    
#########################
########################
    
full_drum_seq1.MergeFrom(full_drum_seq2)

full_drum_seq = full_drum_seq1

with open("pickled_track1_drums", "wb") as f:
    pickle.dump(sequence_proto_to_pretty_midi(full_drum_seq),file=f)

duration = 3.6923076923076925
duration = 7.384615384615385
duration = 11.076923076923077
duration = 14.76923076923077
Using primer[(35,), (), (42,), (), (35,), (), (42,), ()]
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpyulw1ssd/model.ckpt
INFO:tensorflow:Beam search yields sequence with log-likelihood: -12.830133 
number of notes (drum_seq1) = 14
duration = 18.461538461538463
Using primer[(35,), (), (42,), (), (35,), (), (42,), ()]
INFO:tensorflow:Beam search yields sequence with log-likelihood: -12.830133 
number of notes (drum_seq1) = 28
duration = 22.153846153846157
Using primer[(35,), (), (42,), (), (35,), (), (42,), ()]
INFO:tensorflow:Beam search yields sequence with log-likelihood: -12.830133 
number of notes (drum_seq1) = 42
duration = 25.84615384615385
Using primer[(35,), (), (42,), (), (35,), (), (42,), ()]
INFO:tensorflow:Beam search yields sequence with log-likelihood: -12.

In [ ]:
mm.play_sequence(full_drum_seq, mm.midi_synth.fluidsynth)

In [46]:
len(full_drum_seq.notes)

352

In [35]:
pd.DataFrame.from_records([dict(n.ListFields()) for n in list(full_drum_seq.notes)])

,<google.protobuf.pyext._message.FieldDescriptor object at 0x7f6d3c5398f0>,<google.protobuf.pyext._message.FieldDescriptor object at 0x7f6d3c6b4c50>,<google.protobuf.pyext._message.FieldDescriptor object at 0x7f6d3c7a5ad0>,<google.protobuf.pyext._message.FieldDescriptor object at 0x7f6d3c7a5570>,<google.protobuf.pyext._message.FieldDescriptor object at 0x7f6d3c7a5530>,<google.protobuf.pyext._message.FieldDescriptor object at 0x7f6d3c4c85b0>
0,35,100,0.115385,9,True,NaN
1,49,100,0.346154,9,True,0.230769
2,38,100,58.730769,9,True,58.615385
3,38,100,58.846154,9,True,58.730769
4,38,100,58.961538,9,True,58.846154
5,38,100,59.076923,9,True,58.961538
